## Loading Dependencies

In [11]:
%pip install pandas
%pip install matplotlib
%pip install plotly
%pip install scikit-learn
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Importing Libraries

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import plotly.graph_objects as go

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Stage: Preprocessing
##### Loading in the Badminton Serving Data. 
- Blank rows are used to separarate observations. 
- Non-sparse data are filtered out
- Data is grouped into observations labelled under data column "OBSERVATION DATA"
- There are a total of 365 (index 0 - 364) observation groups here
- Data is indicated as serving data with 1 under data column "SERVING"
#### Code is currently commented out as results has already been extracted

In [13]:
# # read csv data from badmintondata.csv
# serving_data = pd.read_csv('badmintondata.csv')

# observation_num = 0
# is_group = False

# # used to filter out blank rows while also labelling data based on their observation groups
# for index, row in serving_data.iterrows():
#     if row['HUMAN PLAYER POSITION (X) metres'] == 0:
#         if is_group:
#             observation_num += 1
#             is_group = False
#         serving_data.drop(index, inplace=True)
#     elif row['HUMAN PLAYER POSITION (X) metres'] == 4 and not is_group:
#         is_group = True
#         serving_data.at[index, 'OBSERVATION NUMBER'] = observation_num
#     elif row['HUMAN PLAYER POSITION (X) metres'] == 4 and is_group:
#         serving_data.at[index, 'OBSERVATION NUMBER'] = observation_num

# serving_data['OBSERVATION NUMBER'] = serving_data['OBSERVATION NUMBER'].astype(int)
# serving_data['SERVING SHOT'] = 1

# # output served data to csv
# rally_data.to_csv('cleaned_served_badminton.csv')

##### Loading in the Badminton Rallying Data. 
- Blank rows are used to separarate observations. 
- Non-sparse data are filtered out
- Data is grouped into observations labelled under data column "OBSERVATION DATA". 
- There are a total of 1212 (index 364 - 1576) observation groups here
- Data is indicated as serving data with 0 under data column "SERVING"
#### Code is currently commented out as results has already been extracted

In [14]:
# # returning shots after the serving shot
# rally_data = pd.read_csv('badmintondata2.csv')
# # sample_rally_data = rally_data.head(1000)
# is_group = False

# for index, row in rally_data.iterrows():
#     if row['HUMAN PLAYER POSITION (X) metres'] == 0:
#         if is_group:
#             observation_num += 1
#             is_group = False
#         rally_data.drop(index, inplace=True)
#     elif row['HUMAN PLAYER POSITION (X) metres'] == 4 and not is_group:
#         is_group = True
#         rally_data.at[index, 'OBSERVATION NUMBER'] = observation_num
#     elif row['HUMAN PLAYER POSITION (X) metres'] == 4 and is_group:
#         rally_data.at[index, 'OBSERVATION NUMBER'] = observation_num

# # ensure observation numbers are integers
# rally_data['OBSERVATION NUMBER'] = rally_data['OBSERVATION NUMBER'].astype(int)
# # indicates if observation is a service shot or not
# rally_data['SERVING SHOT'] = 0

# # output rally data to csv
# rally_data.to_csv('cleaned_rally_badminton.csv')

#### Combining preprocessing
- combine cleaned served and rallied dataframes
- label data if they are are out of bounds within the observations using data column "OUT OF BOUNDS". This includes
    - Data that exceeds the X axis (hit too far on both sides) X < 0 OR X > 13.4
    - Data that exceeds the Y axis (hit out of court on left or right side) Y < 0 OR Y > 5.18
    - Data that is out of bounds labelled as 1, no 0
#### Code is currently commented out as results has already been extracted

In [15]:
# # combining the serving and rally dataframes
# badminton_data = pd.concat([serving_data, rally_data], ignore_index=True)
# # labelling data if its out of bounds
# unique_observation_number = badminton_data['OBSERVATION NUMBER'].unique()
# for obs_number in unique_observation_number:
#     last_row = badminton_data[badminton_data['OBSERVATION NUMBER'] == obs_number].tail(1)
#     # Check the conditions for 'Shuttlecock Position in Air X' and 'Shuttlecock Position in Air Y'
#     if (last_row['SHUTTLECOCK POSITIION IN AIR(X ) metres'].values[0] < 0 or
#         last_row['SHUTTLECOCK POSITIION IN AIR(X ) metres'].values[0] > 13.4 or
#         last_row['SHUTTLECOCK POSITIION IN AIR(Y) metres'].values[0] < 0 or
#         last_row['SHUTTLECOCK POSITIION IN AIR(Y) metres'].values[0] > 5.18):
        
#         # Set 'OUT OF BOUNDS' column to 1 for the corresponding observation number
#         badminton_data.loc[badminton_data['OBSERVATION NUMBER'] == obs_number, 'OUT OF BOUNDS'] = 1
#     else:
#         badminton_data.loc[badminton_data['OBSERVATION NUMBER'] == obs_number, 'OUT OF BOUNDS'] = 0
    
# badminton_data.to_csv('cleaned_combined_data.csv')

#### Sampling of Data

In [20]:
cleaned_data = pd.read_csv('cleaned_badminton.csv')
# removal indexes
cleaned_data = cleaned_data.drop(columns=['Unnamed: 0'])

#### Significance of Independent Features with PCA

In [21]:
# removal of labels and dependent features
test_significance = cleaned_data.drop(columns=['OBSERVATION NUMBER', 
                                               'SERVING SHOT', 
                                               'OUT OF BOUNDS', 
                                               'SHUTTLECOCK POSITIION IN AIR(X ) metres', 
                                               'SHUTTLECOCK POSITIION IN AIR(Y) metres', 
                                               'SHUTTLECOCK POSITIION IN AIR(Z) metres'])

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(test_significance)

pca = PCA()
principal_components = pca.fit_transform(scaled_data)

# Calculate the absolute mean of the loadings for each feature across all principal components
loadings = np.abs(pca.components_)
feature_significance = np.mean(loadings, axis=0)

# Create a DataFrame to display the feature significance
feature_significance_df = pd.DataFrame({'Feature': test_significance.columns, 'Significance': feature_significance})
feature_significance_df = feature_significance_df.sort_values(by='Significance', ascending=False).reset_index(drop=True)

print(feature_significance_df)

                                       Feature  Significance
0  SHUTTELCOCK SLANT ANGLE TO SIDELINE(DEGREE)      0.295305
1        INITITAL VELOCITY OF SHUTTELCOCK(m/s)      0.295173
2    INITIAL SHUTTELCOCK FIRING ANGLE (DEGREE)      0.290432
3             HUMAN PLAYER POSITION (Y) metres      0.290325
4             HUMAN PLAYER POSITION (X) metres      0.200000


In [18]:
# # Assuming you have a DataFrame named 'sample_serving_data' with columns 'OBSERVATION NUMBER',
# # 'SHUTTLECOCK POSITIION IN AIR(X) metres', 'SHUTTLECOCK POSITIION IN AIR(Y) metres',
# # and 'SHUTTLECOCK POSITIION IN AIR(Z) metres'

# # Filter rows starting from row 2
# df_filtered = sample_serving_data.iloc[2:]

# # Get the maximum group_id value
# max_group_id = df_filtered['OBSERVATION NUMBER'].max()

# fig = go.Figure()

# for group_id in range(max_group_id + 1):
#     group_df = df_filtered[df_filtered['OBSERVATION NUMBER'] == group_id]
#     x = group_df['SHUTTLECOCK POSITIION IN AIR(X ) metres']
#     y = group_df['SHUTTLECOCK POSITIION IN AIR(Y) metres']
#     z = group_df['SHUTTLECOCK POSITIION IN AIR(Z) metres']
    
#     fig.add_trace(go.Scatter3d(
#         x=x, y=y, z=z,
#         mode='markers',
#         name=f'Group {group_id}'
#     ))

# fig.update_layout(
#     scene=dict(
#         xaxis=dict(title='X'),
#         yaxis=dict(title='Y'),
#         zaxis=dict(title='Z')
#     ),
#     title='Interactive 3D Plot'
# )
# fig.write_html("interactive_plot.html")

# # fig.show()

In [19]:
# sample_serving_data = serving_data.head(1000)
# # sample_serving_data.head()
# df_filtered = sample_serving_data.iloc[2:]
# max_group_id = df_filtered['OBSERVATION NUMBER'].max()
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# for group_id in range(max_group_id + 1):
#     group_df = df_filtered[df_filtered['OBSERVATION NUMBER'] == group_id]
#     x = group_df['SHUTTLECOCK POSITIION IN AIR(X ) metres']
#     y = group_df['SHUTTLECOCK POSITIION IN AIR(Y) metres']
#     z = group_df['SHUTTLECOCK POSITIION IN AIR(Z) metres']
    
#     ax.scatter(x, y, z)

# ax.set_xlabel('X')
# ax.set_ylabel('Y')
# ax.set_zlabel('Z')

# plt.show()